In [1]:
import numpy as np

from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Embedding
from keras.optimizers import Adam

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
f  = open('small_vocab_en.txt','r')
en = f.read()

f  = open('small_vocab_fr.txt','r')
fr = f.read()

en  = en.split('\n')
fr  = fr.split('\n')

In [3]:
en_tk = Tokenizer(char_level = False)
en_tk.fit_on_texts(en)

fr_tk = Tokenizer(char_level = False)
fr_tk.fit_on_texts(fr)

en_enc = en_tk.texts_to_sequences(en)
fr_enc = fr_tk.texts_to_sequences(fr)

In [4]:
print(en_enc[0])
print(fr_enc[0])

[17, 23, 1, 8, 67, 4, 39, 7, 3, 1, 55, 2, 44]
[35, 34, 1, 8, 67, 37, 11, 24, 6, 3, 1, 112, 2, 50]


In [5]:
print "English: ",en[0]
print "French: ",fr[0]
print "Encoded English: ",en_enc[0]
print "Encoded French : ",fr_enc[0]

English:  new jersey is sometimes quiet during autumn , and it is snowy in april .
French:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
Encoded English:  [17, 23, 1, 8, 67, 4, 39, 7, 3, 1, 55, 2, 44]
Encoded French :  [35, 34, 1, 8, 67, 37, 11, 24, 6, 3, 1, 112, 2, 50]


In [6]:
# def get_max_len(enc):
#     tmp = []
#     for i in enc:
#         tmp.append(len(i))
#     return max(tmp)

# max_french_sequence_length  = get_max_len(fr_enc)
# max_english_sequence_length = get_max_len(en_enc)

max_french_sequence_length   = 10
max_english_sequence_length  = 10

In [7]:
fr_enc = pad_sequences(fr_enc, 10)
en_enc = pad_sequences(en_enc, 10)

In [8]:
print("English VocabSize: ",len(en_tk.word_index))
print("French  VocabSize: ",len(fr_tk.word_index))

('English VocabSize: ', 199)
('French  VocabSize: ', 345)


In [9]:
en_enc = np.reshape(en_enc, (137860,max_english_sequence_length))
fr_enc = np.reshape(fr_enc, (137860,max_french_sequence_length,1))

In [12]:
french_vocab_size  = len(fr_tk.word_index)+1
english_vocab_size = len(en_tk.word_index)+1

model = Sequential()

model.add(Embedding(english_vocab_size, 64, input_length = max_french_sequence_length) )
model.add(GRU(64, return_sequences=True, activation="tanh"))
model.add(TimeDistributed(Dense(french_vocab_size, activation="softmax")))

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = Adam(1e-3), metrics = ['accuracy'])

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 64)            12800     
_________________________________________________________________
gru_2 (GRU)                  (None, 10, 64)            24768     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 346)           22490     
Total params: 60,058
Trainable params: 60,058
Non-trainable params: 0
_________________________________________________________________


# English to French Translator

In [14]:
model.fit(en_enc, fr_enc , batch_size=1024, epochs = 10, validation_split = 0.1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 124074 samples, validate on 13786 samples
Epoch 1/10
124074/124074 [==============================] - 43s 344us/step - loss: 4.5797 - accuracy: 0.1815 - val_loss: 3.6973 - val_accuracy: 0.2507
Epoch 2/10
124074/124074 [==============================] - 45s 363us/step - loss: 2.8861 - accuracy: 0.3631 - val_loss: 2.1796 - val_accuracy: 0.5432
Epoch 3/10
124074/124074 [==============================] - 43s 346us/step - loss: 1.8353 - accuracy: 0.6053 - val_loss: 1.5956 - val_accuracy: 0.6455
Epoch 4/10
124074/124074 [==============================] - 42s 339us/step - loss: 1.4868 - accuracy: 0.6637 - val_loss: 1.3839 - val_accuracy: 0.6828
Epoch 5/10
124074/124074 [==============================] - 41s 327us/step - loss: 1.3342 - accuracy: 0.6844 - val_loss: 1.2756 - val_accuracy: 0.6918
Epoch 6/10
124074/124074 [==============================] - 47s 376us/step - loss: 1.2490 - accurac

In [43]:
fr_index_word = fr_tk.index_word
fr_index_word[0] = ' '

en_index_word = en_tk.index_word
en_index_word[0] = ' '

for ind in range(100):
    
    pred = model.predict(en_enc[ind:ind+1])[0]

    pre = []
    act = []
    inp = []

    for i in pred:
        pre.append(fr_index_word[np.argmax(i)])

    for i in range(10):
        inp.append(en_index_word[en_enc[ind:ind+1][0][i]])
        
    for i in range(10):
        act.append(fr_index_word[fr_enc[ind:ind+1][0][i][0]])

    print '----------------------------------------------------------'
    
    print
    print "English: ",' '.join(inp)
    print "Actual : ",' '.join(act)
    print "Pred   : ",' '.join(pre)
    print

----------------------------------------------------------

English:  sometimes quiet during autumn and it is snowy in april
Actual :  calme pendant l' automne et il est neigeux en avril
Pred   :  parfois calme en automne et il est neigeux en avril

----------------------------------------------------------

English:  chilly during july and it is usually freezing in november
Actual :  généralement froid en juillet et il gèle habituellement en novembre
Pred   :  froid en juillet et il est généralement habituellement en novembre

----------------------------------------------------------

English:  quiet during march and it is usually hot in june
Actual :  calme en mars et il est généralement chaud en juin
Pred   :  calme en mars et il est généralement chaud en juin

----------------------------------------------------------

English:  sometimes mild during june and it is cold in september
Actual :  parfois légère en juin et il fait froid en septembre
Pred   :  parfois doux en juin et il


English:  beautiful during winter but it is never busy in february
Actual :  pendant l' hiver mais il est jamais occupé en février
Pred   :  beau en hiver mais il est jamais occupé en février

----------------------------------------------------------

English:  wonderful during november but it is sometimes hot in september
Actual :  mois de novembre mais il est parfois chaud en septembre
Pred   :  merveilleux en novembre mais il est parfois chaud en septembre

----------------------------------------------------------

English:  pleasant during november but it is never relaxing in july
Actual :  agréable en novembre mais il est jamais relaxant en juillet
Pred   :  agréable en novembre mais il est jamais relaxant en juillet

----------------------------------------------------------

English:  freezing during autumn but it is never busy in june
Actual :  pendant l' automne mais il est jamais occupé en juin
Pred   :  gel en automne mais il est jamais occupé en juin

-------------------

----------------------------------------------------------

English:        they are going to france next june
Actual :        ils vont en france en juin prochain
Pred   :        de vont l' en en automne prochain

----------------------------------------------------------

English:          he likes strawberries oranges and limes
Actual :  il aime les fraises les oranges et les citrons verts
Pred   :        les il les verts et les verts

----------------------------------------------------------

English:  pleasant during winter and it is sometimes wonderful in december
Actual :  pendant l' hiver et il est parfois merveilleux en décembre
Pred   :  agréable en hiver et il est parfois merveilleux en décembre

----------------------------------------------------------

English:  least favorite fruit but the mango is their least favorite
Actual :  fruit préféré moins mais la mangue est leur moins préférée
Pred   :  fruit préféré moins mais la mangue est leur moins préférée

---------------

In [29]:
en_enc[0][1]

67

In [20]:
fr_index_word

{0: ' ',
 1: 'est',
 2: 'en',
 3: 'il',
 4: 'les',
 5: 'mais',
 6: 'et',
 7: 'la',
 8: 'parfois',
 9: 'jamais',
 10: 'le',
 11: "l'",
 12: 'g\xc3\xa9n\xc3\xa9ralement',
 13: 'moins',
 14: 'aim\xc3\xa9',
 15: 'au',
 16: 'fruit',
 17: 'pr\xc3\xa9f\xc3\xa9r\xc3\xa9',
 18: 'agr\xc3\xa9able',
 19: 'froid',
 20: 'son',
 21: 'chaud',
 22: 'de',
 23: 'plus',
 24: 'automne',
 25: 'mois',
 26: '\xc3\xa0',
 27: 'elle',
 28: 'citrons',
 29: 'paris',
 30: 'inde',
 31: 'unis',
 32: '\xc3\xa9tats',
 33: 'france',
 34: 'jersey',
 35: 'new',
 36: 'chine',
 37: 'pendant',
 38: 'pamplemousse',
 39: 'mon',
 40: 'votre',
 41: 'juin',
 42: 'printemps',
 43: 'janvier',
 44: 'hiver',
 45: 'mars',
 46: '\xc3\xa9t\xc3\xa9',
 47: 'mai',
 48: 'septembre',
 49: 'juillet',
 50: 'avril',
 51: 'novembre',
 52: 'd\xc3\xa9cembre',
 53: 'f\xc3\xa9vrier',
 54: 'octobre',
 55: 'aime',
 56: 'ao\xc3\xbbt',
 57: 'merveilleux',
 58: 'relaxant',
 59: 'doux',
 60: 'humide',
 61: 'notre',
 62: 'californie',
 63: 'sec',
 64: 'leu

In [55]:
model.save('Simple_En2Fr.mdl')

In [174]:
pre

['parfois',
 'calme',
 'en',
 'automne',
 'et',
 'il',
 'est',
 'neigeux',
 'en',
 'avril']